In [1]:
!pip install langchain sentence-transformers pypdf2 transformers huggingface_hub
!pip install langchain-community bitsandbytes==0.45.0


In [2]:
from huggingface_hub import login
import os

In [3]:
# Basic import

from torch import cuda, bfloat16
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from time import time
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma

In [4]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineG

In [5]:
!pip install -U transformers
#!pip install -U bitsandbytes

In [6]:
# Loading the Llama2-7b parameter model using 4-bit Quantization

model_id = "meta-llama/Llama-2-7b-chat-hf"

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory

bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

In [7]:
# Setting up the model and tokenizer

time_1 = time()
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
time_2 = time()
print(f"Prepare model, tokenizer: {round(time_2-time_1, 3)} sec.")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Prepare model, tokenizer: 155.988 sec.


In [8]:
time_1 = time()

query_pipeline = transformers.pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        torch_dtype=torch.float16,
        device_map="auto",)

time_2 = time()

print(f"Prepare pipeline: {round(time_2-time_1, 3)} sec.")

Device set to use cuda:0


Prepare pipeline: 0.101 sec.


In [9]:
def test_model(tokenizer, pipeline, prompt_to_test):
    """
    Perform a query
    print the result
    Args:
        tokenizer: the tokenizer
        pipeline: the pipeline
        prompt_to_test: the prompt
    Returns
        None
    """

    time_1 = time()
    sequences = pipeline(
        prompt_to_test,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=200,)
    time_2 = time()
    print(f"Test inference: {round(time_2-time_1, 3)} sec.")
    for seq in sequences:
        print(f"Result: {seq['generated_text']}")

In [10]:
test_model(tokenizer,
           query_pipeline,
           "Please explain what is the significance of Indian Budget every year?")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Test inference: 5.647 sec.
Result: Please explain what is the significance of Indian Budget every year?
 hopefully it will help to understand the impact of budget on the economy and various sectors.

What are your thought on the current budget and what kind of impact do you think it will have on the economy and various sectors.


In [11]:
llm = HuggingFacePipeline(pipeline=query_pipeline)

llm(prompt="Please explain what is the significance of Indian Budget every year, in less than 100 words.")

<ipython-input-11-48687f431c07>:1: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=query_pipeline)
<ipython-input-11-48687f431c07>:3: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  llm(prompt="Please explain what is the significance of Indian Budget every year, in less than 100 words.")
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'Please explain what is the significance of Indian Budget every year, in less than 100 words. Unterscheidung between a budget and a manifesto.\n\nThe Indian Budget is a significant event every'

In [13]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 6.7 MB/s eta 0:00:00


In [14]:
loader = PyPDFLoader("/content/Indian_Budget.pdf")
documents = loader.load()

In [15]:
documents[0:10]

[Document(metadata={'source': '/content/Indian_Budget.pdf', 'page': 0}, page_content='GOVERNMENT OF INDIA\nBUDGET 2024-2025\nSPEECH\nOF\nNIRMALA SITHARAMAN\nMINISTER OF FINANCE\nJuly 23,  2024'),
 Document(metadata={'source': '/content/Indian_Budget.pdf', 'page': 1}, page_content=''),
 Document(metadata={'source': '/content/Indian_Budget.pdf', 'page': 2}, page_content=' \nCONTENTS \n \nPART – A \n Page No. \nIntroduction 1 \nGlobal Context 1 \nInterim Budget 2 \nBudget Theme 2 \nBudget Priorities 2 \n(i) Productivity and resilience in Agriculture  \n(ii) Employment & Skilling \n(iii) Inclusive Human Resource Development and Social Justice  \n(iv) Manufacturing & Services \n(v) Urban Development  \n(vi) Energy Security \n(vii) Infrastructure \n(viii) Innovation, Research & Development \n(ix) Next Generation Reforms \nBudget Estimates 2024-25 20 \n \n \nPART – B \nIndirect taxes 22 \nDirect Taxes  25 \n \nAnnexure to Part-A 31 \nAnnexure to Part-B 36 \n \n  '),
 Document(metadata={'sourc

In [16]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
all_splits = text_splitter.split_documents(documents)

In [17]:
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}

embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

<ipython-input-17-78fd272816e3>:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [19]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 78.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 102.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 93.0 MB/s eta 0:00:00


In [20]:
vectordb = Chroma.from_documents(documents=all_splits, embedding=embeddings, persist_directory="chroma_db")

In [21]:
retriever = vectordb.as_retriever()

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    verbose=True
)

In [22]:
def test_rag(qa, query):
    print(f"Query: {query}\n")
    time_1 = time()
    result = qa.run(query)
    time_2 = time()
    print(f"Inference time: {round(time_2-time_1, 3)} sec.")
    print("\nResult: ", result)

In [23]:
query = "Please explain what is the significance of Indian Budget every year?"
test_rag(qa, query)

Query: Please explain what is the significance of Indian Budget every year?



> Entering new RetrievalQA chain...


<ipython-input-22-b2e5efb36925>:4: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa.run(query)
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



> Finished chain.
Inference time: 7.804 sec.

Result:  Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

over a 5-year period with a central outlay of ` 2 lakh crore.  I will speak about 
them shortly, while more details may be seen in the annexure.  This year, I 
have made a provision of ` 1.48 lakh crore for education, employment and 
skilling.     
Budget Priorities 
7. The people have given a unique opportunity to  our government to 
take the country on the path of strong development and all-round prosperity. 
In the interim budget, we promised to present a detailed roadmap for our 
pursuit of ‘ Viksit Bharat’. In line with the strategy set out in the interim 
budget, this budget envisages sustained efforts on the following 9 priorities 
for generating ample opportunities for all.   
1) Productivity and resilience in Agriculture 
2) Employment & Skilling 
3) Inclusive 

In [24]:
query = "What are the key highlights of the Indian Budget 2024-2025?"
test_rag(qa, query)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Query: What are the key highlights of the Indian Budget 2024-2025?



> Entering new RetrievalQA chain...

> Finished chain.
Inference time: 6.745 sec.

Result:  Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Budget 2024-2025 
 
Speech of 
Nirmala Sitharaman 
Minister of Finance 
July 23, 2024 
Hon’ble Speaker,  
 I present the Budget for 2024-25.  
Introduction 
1. The people of India have reposed their faith in the government led by 
the Hon’ble Prime Minister Shri Narendra Modi and re-elected it for a historic 
third term under his leadership. We are grateful for their support, faith and 
trust in our policies. We are determined to ensure that all Indians, regardless 
of religion, caste, gender and age, make substantial progress in realising their 
life goals and aspirations. 
Global Context 
2. The global economy, while performing better than expected, is still in 
t

In [25]:
docs = vectordb.similarity_search(query)
print(f"Query: {query}")
print(f"Retrieved documents: {len(docs)}")
for doc in docs:
    doc_details = doc.to_json()['kwargs']
    print("Source: ", doc_details['metadata']['source'])
    print("Text: ", doc_details['page_content'], "\n")
    break

Query: What are the key highlights of the Indian Budget 2024-2025?
Retrieved documents: 4
Source:  /content/Indian_Budget.pdf
Text:  Budget 2024-2025 
 
Speech of 
Nirmala Sitharaman 
Minister of Finance 
July 23, 2024 
Hon’ble Speaker,  
 I present the Budget for 2024-25.  
Introduction 
1. The people of India have reposed their faith in the government led by 
the Hon’ble Prime Minister Shri Narendra Modi and re-elected it for a historic 
third term under his leadership. We are grateful for their support, faith and 
trust in our policies. We are determined to ensure that all Indians, regardless 
of religion, caste, gender and age, make substantial progress in realising their 
life goals and aspirations. 
Global Context 
2. The global economy, while performing better than expected, is still in 
the grip of policy uncertainties. Elevated asset prices, political uncertainties 
and shipping disruptions continue to pose significant downside risks for 
growth and upside risks to inflation. 